In [ ]:
conda install -c rdkit rdkit

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
import pandas as pd
import numpy as np
import rdkit
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors as desc
from rdkit.Chem import rdqueries

In [ ]:
import os
os.getcwd()

In [ ]:
data=pd.read_csv('/kaggle/input/chemical-structure-and-logp/logP_dataset.csv', header=None)
data.columns=['SMILES','LogP']

In [ ]:
data['Formula']=data['SMILES'].apply(lambda x: desc.CalcMolFormula(Chem.MolFromSmiles(x)))

In [ ]:
for i in range(1,54):
    q = rdqueries.AtomNumEqualsQueryAtom(i)
    data['AtNo'+str(i)]=data['SMILES'].apply(lambda x: len(Chem.AddHs(Chem.MolFromSmiles(x)).GetAtomsMatchingQuery(q)))

In [ ]:
data.head(10)

In [ ]:
data.sum()

In [ ]:
Atoms = ['AtNo'+str(i) for i in range(1,54) if sum(data['AtNo'+str(i)])>0 ]

In [ ]:
Atoms

In [ ]:
import statsmodels.api as sm
X = data[Atoms]
y = data["LogP"]
model = sm.OLS(y, X).fit()
data["pred_LogP"] = model.predict(X)

In [ ]:
import seaborn as sns
from scipy import stats
def r2(x, y):
    return stats.pearsonr(x, y)[0] ** 2

sns.jointplot(x=data["pred_LogP"],
            y=data["LogP"], 
            kind="reg", 
            stat_func=r2,
            scatter_kws={"s": 1});

In [ ]:
data['rings']=data['SMILES'].apply(lambda x: desc.CalcNumRings(Chem.MolFromSmiles(x)))

In [ ]:
X = data[Atoms+['rings']]
y = data["LogP"]
model = sm.OLS(y, X).fit()
data["pred_LogP2"] = model.predict(X)


In [ ]:
sns.jointplot(x=data["pred_LogP2"],
            y=data["LogP"], 
            kind="reg", 
            stat_func=r2,
            scatter_kws={"s": 1});

In [ ]:
r2(x=data["pred_LogP"],y=data["LogP"])

In [ ]:
r2(x=data["pred_LogP2"],y=data["LogP"])

In [ ]:
data['LogP_diff']=abs(data["pred_LogP2"]-data["LogP"])

In [ ]:
data.sort_values('LogP_diff').tail(20)
